# Project 2: Topic Classification

W207  <br />
Class: Thursday 6:30-8 PDT <br />
Student: Clayton Leach

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [65]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

#Changed to .model_selection as sklearn moved things around
from sklearn.model_selection import GridSearchCV
#Including for Grid search (I know we aren't supposed to add packages!)
from sklearn.metrics import make_scorer, accuracy_score

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [3]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)


# Added round function to index selection as len(newsgroups_test) is odd
test_data, test_labels = newsgroups_test.data[int(round(num_test/2,0)):], \
                                            newsgroups_test.target[int(round(num_test/2,0)):]
dev_data, dev_labels = newsgroups_test.data[:int(round(num_test/2,0))], \
                                            newsgroups_test.target[:int(round(num_test/2,0))]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print ('training label shape:', train_labels.shape)
print ('test data shape:', len(test_data))
print ('test label shape:', test_labels.shape)
print ('dev data shape:', len(dev_data))
print ('dev label shape:', dev_labels.shape)
print ('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test data shape: 677
test label shape: (677,)
dev data shape: 676
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [4]:
def print_training_examples(examples, labels, label_key, num_examples=5):
    for x in range(0,num_examples):
        print("Example %s is of type %s and reads:" % ((x+1),categories[labels[x]]))
        print("")
        print(examples[x])
        print("")
    

print_training_examples(train_data, train_labels, categories)


Example 1 is of type talk.religion.misc and reads:

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Example 2 is of type sci.space and reads:



Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating s

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [5]:
def vectorize_analysis(train_data):
    vectorize = CountVectorizer()
    features = vectorize.fit_transform(train_data)
    return features, vectorize

def tfidf_vectorize_analysis(train_data, vocab=[], ngram_selection = (1,1), analyzer = 'word', min_doc_freq=1):
    if len(vocab) <1:
        vectorize = TfidfVectorizer(ngram_range=ngram_selection, analyzer=analyzer,
                                    min_df=min_doc_freq)
    else:
        vectorize = TfidfVectorizer(vocabulary=vocab, ngram_range=ngram_selection, analyzer=analyzer,
                                   min_df=min_doc_freq)
        
    features = vectorize.fit_transform(train_data)
    
    return features, vectorize

feature_matrix, vectorize = vectorize_analysis(train_data)


# Part a
print("The vocabulary length is equal to the number of columns, which is:",feature_matrix.shape[1])
print("The average non-zero values per example is equal to the total non-zero values divided by"
      "the number of examples: %s / %s = %s" %(feature_matrix.nnz,feature_matrix.shape[0],
                                               feature_matrix.nnz/feature_matrix.shape[0]))
print()


# Part b
feature_names = vectorize.get_feature_names()
alpha_feature_names = [feature for feature in feature_names if feature.isalpha()]
print("The first string is", feature_names[0])
print("The first string with only letters is",alpha_feature_names[0])
print("The last string is", feature_names[len(feature_names)-1])
print()


# Part c
vocab = ["atheism", "graphics", "space", "religion"]

# get vocab specific (vs) feature matrix and vectorizer object
vs_feature_matrix, vs_vectorize = tfidf_vectorize_analysis(train_data, vocab=vocab)
print("The vocabulary length is equal to the number of columns, which is:",vs_feature_matrix.shape[1])
print("The average non-zero values per example is equal to the total non-zero values divided by"
      "the number of examples: %s / %s = %s" %(vs_feature_matrix.nnz,vs_feature_matrix.shape[0],
                                               vs_feature_matrix.nnz/vs_feature_matrix.shape[0]))
print()


# Part d
# Get feature matrix and vectorizer object including n-grams
for i in range(1,4):
    ng_feature_matrix, ng_vectorize = tfidf_vectorize_analysis(train_data, ngram_selection=(1,i))
    print("Vocab length when we use combinations up to %s-grams is:" %i, ng_feature_matrix.shape[1])
print()
    
    
#Part e
# Get feature matrix and vectorizer object with pruning if word in less than 10 documents
pruned_feature_matrix, pruned_vectorize = tfidf_vectorize_analysis(train_data, min_doc_freq=10)
print("The vocabulary length is equal to the number of columns, which is:",pruned_feature_matrix.shape[1])
print()

#Part f
dev_feature_matrix, dev_vectorize = vectorize_analysis(dev_data)
dev_dict = dev_vectorize.get_feature_names()
train_dict = vectorize.get_feature_names()
missing_dev_words = [word for word in dev_dict if word not in train_dict]
fraction_dev_missing = len(missing_dev_words)/len(dev_dict)
print("%s percent of the words in the dev vocab are not in the training vocab" %round((fraction_dev_missing*100),2))

The vocabulary length is equal to the number of columns, which is: 26879
The average non-zero values per example is equal to the total non-zero values divided bythe number of examples: 196700 / 2034 = 96.70599803343165

The first string is 00
The first string with only letters is aah
The last string is zyxel

The vocabulary length is equal to the number of columns, which is: 4
The average non-zero values per example is equal to the total non-zero values divided bythe number of examples: 546 / 2034 = 0.26843657817109146

Vocab length when we use combinations up to 1-grams is: 26879
Vocab length when we use combinations up to 2-grams is: 221770
Vocab length when we use combinations up to 3-grams is: 537462

The vocabulary length is equal to the number of columns, which is: 3064

24.79 percent of the words in the dev vocab are not in the training vocab


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [45]:
# If we use the data sets independently we will get mismatched feature lengths because some words
# appear in one set, but not in the other.  To solve for this, I first combine the dev and training
# vocabularys to create a master vocabulary.  

test_feature_matrix, test_vectorize = vectorize_analysis(test_data)
dev_feature_matrix, dev_vectorize = vectorize_analysis(dev_data)

total_vocab={}
for k,v in test_vectorize.vocabulary_.items():
    total_vocab[k]=v

for k,v in dev_vectorize.vocabulary_.items():
    total_vocab[k]=v

no_dupes_vocab = []
for k,v in total_vocab.items():
    no_dupes_vocab.append(k)
    
test_feature_matrix, test_vectorize = tfidf_vectorize_analysis(test_data,vocab = no_dupes_vocab)
dev_feature_matrix, dev_vectorize = tfidf_vectorize_analysis(dev_data, vocab = no_dupes_vocab)

In [74]:
def knn(k_value,features,labels,dev_features,dev_labels):
    """
    fit a knn model, and report model features on a dev set
    
    :param k_values: hyperparameter indicating number of neighbors for model to consider
    :param features: training feature set
    :param labels: training label set
    :param dev_features: dev feature set
    :param dev_labels: dev label set
    """
    
    knn_model = KNeighborsClassifier(n_neighbors=k_value)
    knn_model.fit(features,labels)
    
    #knn "score" is accuracy, so don't need to specify accuracy_score
    accuracy = knn_model.score(dev_features,dev_labels)
    
    return knn_model, accuracy


k=2
knn_model, accuracy = knn(k,test_feature_matrix,test_labels,dev_feature_matrix,dev_labels)

dev_predictions = knn_model.predict(dev_feature_matrix)
knn_f1_score = metrics.f1_score(dev_labels,dev_predictions, average='micro')

In [90]:
def k_grid_search(k_values,scoring_method,test_features, test_labels):
    """
    Searches for optimal k value for a knn model and returns the 
    output from the GridSearchCV function
    
    :param k_values: list of k values to try
    :scoring_method: scoring method to be used in GridSearch
    """
    
    parameters = {'n_neighbors':k_values}
    scorer = make_scorer(scoring_method)
    knn_model = KNeighborsClassifier()
     
    best_k_finder = GridSearchCV(knn_model, parameters,
                                    scoring=scorer,return_train_score=True)
    gs_output = best_k_finder.fit(test_features, test_labels)
    return gs_output


k_vals = [x for x in range(1,11)]
best_knn = k_grid_search(k_vals, accuracy_score, test_feature_matrix, test_labels)
best_knn.cv_results_

/Users/clayleach/p3env/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/clayleach/p3env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The optimal value of k appears to be 7 based on the mean test results.

Part a: KNN doesn't work well for this problem because so little of our space is actually covered.  Space grows exponentially as you add dimensions (e.g. if you assume any dimension can go from 0-100 in integers, then going from 1 dimension to 2 dimensions increases possible combinations from 100, to 100^2) and our feature space is over 10,000.  At this size we would need an absolutely monsterous corpus of texts (maybe larger than exists?) in order to realistically expect new examples to have neighbors that are in fact "near".

ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [6]:
#def P4():
### STUDENT START ###


### STUDENT END ###
#P4()

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [7]:
def empty_preprocessor(s):
    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

    
### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [8]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    

    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [11]:
#def P7():
    ### STUDENT START ###



    ### STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance